In [1]:
%load_ext autoreload
import race
import math
import random

import gym
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.distributions import Normal

from IPython.display import clear_output
import matplotlib.pyplot as plt
%matplotlib inline

pygame 2.5.2 (SDL 2.28.3, Python 3.11.3)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
use_cuda = torch.cuda.is_available()
device   = torch.device("cuda" if use_cuda else "cpu")
device

device(type='cuda')

In [3]:
env = race.CurvyRace()

In [40]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

# Define the deep Q-learning agent
class DQNAgent(nn.Module):
    def __init__(self, state_size, action_size):
        super(DQNAgent, self).__init__()
        self.state_size = state_size
        self.action_size = action_size

        self.fc1 = nn.Linear(state_size, 255)
        self.fc2 = nn.Linear(255, 255)
        self.fc3 = nn.Linear(255, 255)
        self.fc4 = nn.Linear(255, 255)

        self.fc5 = nn.Linear(255, action_size)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = torch.relu(self.fc4(x))
        x = self.fc5(x)
        return x

    def act(self, state):
        # Epsilon-greedy policy
        if np.random.rand() <= 0.1:
            return torch.tensor(np.random.uniform(low=-1, high=1, size=self.action_size), dtype=torch.float32)
        with torch.no_grad():
            state = torch.tensor(state, dtype=torch.float32)
            q_values = self(state)
            return q_values

    def replay(self, state, action, reward, next_state, done, optimizer, criterion):
        state = torch.tensor(state, dtype=torch.float32)
        action = torch.tensor(action, dtype=torch.float32)
        reward = torch.tensor(reward, dtype=torch.float32)
        next_state = torch.tensor(next_state, dtype=torch.float32)

        target = self(state).clone()
        if done:
            target[np.argmax(action)] = reward
        else:
            with torch.no_grad():
                t = self(next_state).numpy()
                target[np.argmax(action)] = reward + 0.95 * np.max(t)

        loss = criterion(self(state), target)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()


# Define the training parameters
state_size = 3  # observation vector dimension
action_size = 2  # action vector dimension
agent = DQNAgent(state_size, action_size)

optimizer = optim.Adam(agent.parameters(), lr=0.05)
criterion = nn.MSELoss()


# Training loop
actions = []
num_episodes = 1000
target_update_frequency = 10
for episode in range(num_episodes):
    state = env.reset()
    total_reward = 0

    for t in range(env.get_max_steps()):
        action = agent.act(state)
        actions.append(action)
        if episode % target_update_frequency == 0:
            next_state, reward, done = env.step(action)
            total_reward += reward
    
            agent.replay(state, action, reward, next_state, done, optimizer, criterion)
    
            if done:
                break
    
            state = next_state
        else:                   
            next_state, reward, done = env.step(action)
            total_reward += reward
            
            if done:
                break
    
            state = next_state
            
        

    if episode % 100 == 0:
        #env.plot()
        print("Episode: {}, Total Reward: {}".format(episode, total_reward))

    # Update target model weights every target_update_frequency episodes
    if episode % target_update_frequency == 0:
        agent.load_state_dict(agent.state_dict())

# Test the trained agent
state = env.reset()
done = False
while not done:
    action = agent.act(state)
    next_state, _, done = env.step(action)
    env.plot()
    state = next_state

#plt.show()


C:\Users\Anwender\AppData\Local\Temp\ipykernel_12136\454096615.py:39: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  action = torch.tensor(action, dtype=torch.float32)


Episode: 0, Total Reward: 0
Episode: 100, Total Reward: 0


KeyboardInterrupt: 

In [35]:
actions